# 爱奇艺弹幕解析

## 解析 XML 文件
弹幕的存储格式为
```xml
<data>
	<entry>
		<int>1</int>
		<list>
			<bulletInfo>
				<contentId>1417507431223347040</contentId>
				<content>哪句话能</content>
				<showTime>1</showTime>
				<font>30</font>
				<color>FFFFFF</color>
				<opacity>0</opacity>
				<position>0</position>
				<background>0</background>
				<contentType>0</contentType>
				<isReply>false</isReply>
				<userInfo>
					<uid>2059497678</uid>
					<udid>89ce7ce0600eb652b6189a0cbef7967e</udid>
					<name>null</name>
				</userInfo>
			</bulletInfo>
		</list>
	</entry>
    ...
<data>
```

In [ ]:
from xml.etree.ElementTree import parse
def parse_dm_xml(dm_xml,episode,episode_zh,dm_csv):
    with open(dm_xml, 'rb') as f:
        doc = parse(dm_xml)
        
        # 解析弹幕
        for item in doc.iterfind('data/entry/list/bulletInfo'):
    #         uid=""
    #         udid=""
            for user in item.iterfind('userInfo'):
                uid = user.findtext('uid')
                udid = user.findtext('udid')

            contentId = item.findtext('contentId')
            content = item.findtext('content')
            showTime = item.findtext('showTime')

            dm_csv.writerow((uid , udid , contentId , content, showTime , episode, episode_zh))

In [ ]:
import os
import csv
from tqdm import tqdm 
def parse_danmu(file_path):
    # 创建存储弹幕信息的文件
    dm = open(r'/Users/weduoo/Desktop/wlwz_dm.csv', 'w')
    dm_csv = csv.writer(dm)
    # 添加 csv 头部信息
    dm_csv.writerow(["uid","udid","contentId","content","showTime","episode","episode_zh"])
    # 读取存储每集弹幕的文件夹
    dm_dirs = os.listdir(file_path)
    
    for i,dm_dir in tqdm(enumerate(dm_dirs)):
        if not dm_dir.startswith('.'):
            # 表示第几集
            if i < 9:
                episode = "episode_0" + str(i + 1)
                episode_zh = "第 0" + str(i + 1) + " 集"
            else:
                episode = "episode_" + str(i + 1)
                episode_zh = "第 " + str(i + 1) + " 集"
            
            dm_xmls = os.listdir(file_path + "/" + dm_dir)

            for dm_xml in dm_xmls:
                if not dm_xml.startswith('.'):
                    # 读取 xml 文件
                    dm_xml = file_path + "/" + dm_dir + "/" + dm_xml
                    parse_dm_xml(dm_xml,episode,episode_zh,dm_csv)
    dm.close()

In [ ]:
parse_danmu("/Users/weduoo/Desktop/wlwz")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from pyecharts import Bar
from pyecharts import online

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 让 pyecharts 在 jupyter 下载时有图片
online()

In [ ]:
data = pd.read_csv("/Users/weduoo/Desktop/wlwz_dm.csv")
data.head()

In [ ]:
y = data.groupby("episode_zh").size()
bar = Bar("每集弹幕数量",width=950)
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
data.groupby("udid")["udid"].value_counts().max()

In [ ]:
# 筛选出含有李小璐的弹幕
lxl = data[data["content"].str.contains("李小璐")]
print("总评论数：",len(lxl))
print("参与的人数", len(lxl.groupby("udid")["udid"].value_counts()))
lxl
y = lxl.groupby("episode_zh").size()
print("出现的集数", len(list(y.index)))
bar = Bar("出现李小璐的集数及其次数",width=950)
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
# 筛选出含有李小璐的弹幕
lxl = data[data["content"].str.contains("霍金")]
print("总评论数：",len(lxl))
print("参与的人数", len(lxl.groupby("udid")["udid"].value_counts()))
lxl
y = lxl.groupby("episode_zh").size()
print("出现的集数", len(list(y.index)))
bar = Bar("出现李小璐的集数及其次数",width=950)
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
# 筛选出含有李小璐的弹幕
lxl = data[data["content"].str.contains("哈哈")]
print("总评论数：",len(lxl))
print("参与的人数", len(lxl.groupby("udid")["udid"].value_counts()))
lxl
y = lxl.groupby("episode_zh").size()
print("出现的集数", len(list(y.index)))
bar = Bar("出现李小璐的集数及其次数",width=950)
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
def create_wordcloud(text):
    # 用于支持中文的设置
    font = '/Library/Fonts/Songti.ttc'
    wordcloud = WordCloud(background_color="white",font_path=font,\
                          width=1000, height=1000, margin=2).generate(text)
    # 生成图片
    plt.figure(figsize=(14,14))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
dm_text = [] 
for text in list(data["content"]):
    t = text.strip().replace("哈","")
    dm_text.append(t.strip())
create_wordcloud("\n".join(dm_text))

# 乐视甄嬛传弹幕与评论解析`

**以下信息均可以去掉 jQuery 参数**

剧集信息 url：http://d-api-m.le.com/card/dynamic?platform=pc&callback=jQuery191011019487064499534_1525083223860&vid=1559865&cid=2&id=73868&pagesize=15&type=episode&isvip=0&page=1&_=1525083223861
```
```

基础信息 url：http://v.stat.le.com/vplay/queryMmsTotalPCount?callback=jQuery19105577507525270364_1525079352904&vid=1559584&_=1525079352921
```
{
    "vdm_count": 23544, 
    "is_comm": 1, 
    "vreply": 2285, 
    "vcomm_count": 7656, 
    "vnpcomm": 7120, 
    "media_play_count": 21326745, 
    "is_dm": 1, 
    "up": 74082, 
    "down": 23193
}
```

弹幕请求 url：http://cdn.api.my.le.com/danmu/list?vid=1559584&cid=2&start=0&amount=2000&getcount=1
```json
{
    "code": 200, 
    "data": {
        "list": [], 
        "user": [ ], 
        "vidDanmuCount": "18506"
    }
}
 {
    "uid": 291036303, 
    "start": 142.556, 
    "txt": "齐妃上海人吧", 
    "color": "FFFFFF", 
    "font": "m", 
    "type": "txt", 
    "position": 2, 
    "addtime": 1524814538, 
    "from": 2, 
    "vip": 3, 
    "version": "7.12", 
    "region": "CN", 
    "appid": 0, 
    "sappid": 0, 
    "platid": 0, 
    "_id": "5248145388932101016"
}
```



评论请求 url：http://api.my.le.com/vcm/api/list?jsonp=jQuery19105577507525270364_1525079352904&type=video&rows=20&page=2&sort=&cid=2&source=1&xid=1559584&pid=73868&ctype=cmt%2Cimg%2Cvote&listType=1&_=1525079352922
```json
{
    "result": "200", 
    "total": 7656, 
    "replyTotal": 2285, 
    "data": [], 
    "rule": 1, 
    "authData": [ ], 
    "topData": [ ], 
    "godData": [ ], 
    "announcementData": [ ], 
    "version_time": "2018-04-30 16:00:27"
}
{
     "_id": "5245677405075101068", 
     "content": "自插双眼？", 
     "vtime": "5天前", 
     "ctime": "1524567740", 
     "city": "北京", 
     "replynum": 0, 
     "voteFlag": "0", 
     "ip": "3698331511", 
     "like": 3, 
     "img": false, 
     "imgPack": [ ], 
     "htime": 0, 
     "pid": "73868", 
     "xid": "1559584", 
     "lid": "101", 
     "cid": 2, 
     "replys": [ ], 
     "source": {
         "id": 3, 
         "detail": "乐视网"
     }, 
     "cmtType": "cmt", 
     "vote": [ ], 
     "votenum": 0, 
     "status": "2", 
     "user": {
         "uid": "173369259", 
         "ssouid": "173369259", 
         "username": "广东乐迷", 
         "photo": "http://i2.letvimg.com/lc01_iscms/201510/13/16/56/ab665c35af9d47beb2f7078cbaf308de.png", 
         "isvip": 0, 
         "cooperation": [ ]
     }, 
     "isLike": false, 
     "isVoted": false
}, 
```

In [ ]:
import urllib
from urllib import request
import json
from retrying import retry
import os
import csv
from tqdm import tqdm

## 评论解析

In [ ]:

"""
获取剧集信息
episode_nums:当前电视剧有多少集
episode:当前电视剧 ID
"""
def get_episode(episode, episode_nums):
    url = "http://d-api-m.le.com/card/dynamic?platform=pc&vid=" + str(episode) + "&cid=2&id=73868&pagesize="+str(episode_nums)+"&type=episode&isvip=0&page=1"
    rep = request.urlopen(url)
    return json.loads(rep.read())

"""
获取基础评论信息
episode_id:集 id
""" 
def get_base_info(episode_id):
    url = "http://v.stat.le.com/vplay/queryMmsTotalPCount?vid=" + str(episode_id)
    rep = request.urlopen(url)
    return json.loads(rep.read())
 
"""
获取评论信息
vcomm_count:评论数量
episode_id:集 id
"""
@retry
def comm_get(vcomm_count,page,episode_id,pid):
    url="http://api.my.le.com/vcm/api/list?type=video&rows="+str(vcomm_count)+"&page="+str(page)+"&sort=&cid=2&source=1&xid="+str(episode_id) +"&pid="+str(pid)+"&listType=1"
    rep = request.urlopen(url)
    return json.loads(rep.read())

"""
解析评论
comm_csv:写入 csv 
datas:评论数据
episode:剧集信息
""" 
def parse_comm(comm_csv, datas, episode):
    for data in datas:
        if ("replys" in data.keys()) and len(data["replys"]) > 0:
            parse_comm_sub(comm_csv, data["replys"], episode)
        # 用户
        user = data["user"]
        uid = user["uid"]
        username = user["username"]
        isvip = user["isvip"]
        ip = data["ip"]
        city = data["city"]

        # 评论
        content = data["content"]
        ctime = data["ctime"]
        like = data["like"]
        replynum = 0
        if "replynum" in data.keys():
            replynum = data["replynum"]
        
        # 来源
        source = data["source"]["detail"]

        # 剧集信息
        title = episode["title"]
        subtitle = episode["subTitle"]
        play_count = episode["play_count"]
        
        comm_csv.writerow((uid, username, isvip, ip, city, content, ctime, like, replynum, source, title, subtitle, play_count))

def parse_comm_sub(comm_csv, datas, episode):
    for data in datas:
        if ("replys" in data.keys()) and len(data["replys"]) > 0:
            parse_comm_sub(comm_csv, data["replys"], episode)
        # 用户
        user = data["user"]
        uid = user["uid"]
        username = user["username"]
        isvip = user["isvip"]
        ip = data["ip"]
        city = data["city"]

        # 评论
        content = data["content"]
        ctime = data["ctime"]
        like = data["like"]
        replynum = 0
        if "replynum" in data.keys():
            replynum = data["replynum"]
        
        # 来源
        source = data["source"]["detail"]

        # 剧集信息
        title = episode["title"]
        subtitle = episode["subTitle"]
        play_count = episode["play_count"]
        
        comm_csv.writerow((uid, username, isvip, ip,city, content, ctime, like, replynum, source, title, subtitle, play_count))      

In [ ]:
def create_comm(comm_io,episode_id,episode_nums,pid):
    comm_csv = csv.writer(comm_io)
    # 添加 csv 头部信息
    comm_csv.writerow(["uid","username","isvip", "ip","city","content","ctime","like","replynum","source","title","subtitle","play_count"])
    episodes_data = get_episode(episode_id,episode_nums)
    episodes = episodes_data["data"]["episode"]["videolist"]
    for episode in tqdm(episodes):
        vid = episode["vid"]
        base_info = get_base_info(vid)
        vcomm_count = base_info["vcomm_count"]
        # 一次最多 60 条
        for i in range((vcomm_count//60) + 1 ):
            comms = comm_get(60, i + 1,vid,pid)
            parse_comm(comm_csv,comms["data"],episode)
    
# 创建存储弹幕信息的文件
comm = open(r'/Users/weduoo/Desktop/zhz_comm.csv', 'a')
create_comm(comm,1559865,76,73868)
comm.close()

## 弹幕解析

In [ ]:
"""
获取弹幕信息
vcomm_count:评论数量
episode_id:集 id
"""
@retry
def dm_get(start,episode_id):
    url = "http://cdn.api.my.le.com/danmu/list?vid="+str(episode_id)+"&cid=2&start="+str(start)+"&amount=2000"
    rep = request.urlopen(url)
    return json.loads(rep.read())

"""
解析弹幕
comm_csv:写入 csv 
datas:评论数据
episode:剧集信息
""" 
def parse_dm(dm_csv, datas, episode):
    for data in datas:
        uid = data["uid"]
        addtime = data["addtime"]
        version = data["version"]
        txt = data["txt"]
        types = data["type"]
        title = episode["title"]
        subtitle = episode["subTitle"]
        
        dm_csv.writerow((uid, addtime, version, txt, types, title, subtitle))


In [ ]:
def create_dm(dm_io,episode_id,episode_nums):
    dm_csv = csv.writer(dm_io)
    # 添加 csv 头部信息
    dm_csv.writerow(["uid","addtime","version","txt","type","title","subtitle"])
    episodes_data = get_episode(episode_id,episode_nums)
    episodes = episodes_data["data"]["episode"]["videolist"]
    for episode in tqdm(episodes):
        vid = episode["vid"]
        base_info = get_base_info(vid)
        vdm_count = base_info["vdm_count"]
        
        # 一次最多 60 条
        for i in range(0,25,3):
            dms = dm_get(i * 100,vid)
            parse_dm(dm_csv, dms["data"]["list"], episode)
    
# 创建存储弹幕信息的文件
dm = open(r'/Users/weduoo/Desktop/zhz_dm.csv', 'a')
create_dm(dm,1559865,76)
dm.close()

## 弹幕分析

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from pyecharts import Bar
from pyecharts import online

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 让 pyecharts 在 jupyter 下载时有图片
online()

In [ ]:
data = pd.read_csv("/Users/weduoo/Desktop/zhz_dm.csv")
data.head()

In [ ]:
y = data.groupby("title").size()
bar = Bar("每集弹幕数量",width=950)
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
def create_wordcloud(text):
    # 用于支持中文的设置
    font = '/Library/Fonts/Songti.ttc'
    wordcloud = WordCloud(background_color="white",font_path=font,\
                          width=1000, height=1000, margin=2).generate(text)
    # 生成图片
    plt.figure(figsize=(14,14))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
import re
dm_text = [] 
def remove_words(txt):
    words = ["哈","呵","啦","啊","皇上","甄","嬛","弹幕呢"]
    for word in words:
        txt = txt.replace(word,"")
    return txt
    
for text in tqdm(list(data["txt"])):
#     t = str(text).strip().replace("哈","").replace("呵","").replace("啦","").replace("啊","")
    t = remove_words(str(text))
    t = re.sub('[a-zA-Z]','', t)
    dm_text.append(t.strip())
create_wordcloud("\n".join(dm_text))

## IP 地址转化服务
参考：
- [python整数与IP地址转换 [转]](http://www.cnblogs.com/vovlie/archive/2012/10/17/2727029.html)

ip 地址查询 url：http://ip.taobao.com/service/getIpInfo2.php?ip=8.8.8.8

In [ ]:
# 整数转 IP
def int2ip(x):
    ch2 = lambda x: '.'.join([str(x//(256**i)%256) for i in range(3,-1,-1)])
    return ch2(x)

# ip 转整数
def ip2int(x):
    ch3 = lambda x:sum([256**j*int(i) for j,i in enumerate(x.split('.')[::-1])])
    return ch3(x)
def int2ip2location(x):
    ip = int2ip(x)
    url = "http://ip.taobao.com/service/getIpInfo2.php?ip=" + str(ip)
    rep = request.urlopen(url)
    data = json.loads(rep.read())["data"]
    if data["region"] != "XX":
        return data["region"]
    else:
        return data["country"]

In [ ]:
int2ip2location(-556216572)

In [ ]:
int2ip(-556216572)

In [ ]:
ip2int('222.216.207.4')

## 评论分析

In [ ]:
comms = pd.read_csv("/Users/weduoo/Desktop/zhz_comm.csv")
comms.head()

In [ ]:
comms[comms["ip"] < 0].head()

In [ ]:
ip_citys = {}
ips = list(set(comms["ip"]))
for ip in tqdm(ips):
    ip_citys[ip] = int2ip2location(ip)

In [ ]:
comms["ip_city"] = comms["ip"]
tqdm(comms["ip_city"].apply(int2ip2location))
comms.head()

In [ ]:
y = comms.groupby("title").size()
bar = Bar("每集评论数量",width=950)
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
provinces = ["北京","天津","上海"," 重庆","黑龙江","吉林","辽宁","河北"," 山西"," 青海",\
         "山东","河南","江苏","安徽","浙江","福建","江西","湖南","湖北","广东","台湾","海南",\
         " 甘肃","陕西"," 四川"," 贵州","云南"]

In [ ]:
citys = comms["city"].str.replace("省","")
citys = citys.str.replace("北京市","北京")
citys = citys[citys.isin(provinces)]
y = citys.value_counts()
bar = Bar( str(len(y.index)) + "个地区评论数分布",width=950)
print(list(y.index))
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
from pyecharts import Pie
pie = Pie(title_pos='center',height=550)
pie.add("", list(y.index), y, is_label_show=True,legend_pos='left',is_legend_show=False,legend_orient='vertical',radius=[40, 75])
pie

In [ ]:
import re
comms_text = [] 
def remove_words(txt):
    words = ["哈","呵","啦","啊","皇上","甄","嬛","弹幕呢"]
    for word in words:
        txt = txt.replace(word,"")
    return txt
    
for text in tqdm(list(comms["content"])):
    t = remove_words(str(text))
    t = re.sub('[a-zA-Z]','', t)
    comms_text.append(t.strip())
create_wordcloud("\n".join(comms_text))

# 欢乐颂弹幕及其评论解析

In [ ]:
def create_comm(comm_io,episode_id,episode_nums,pid):
    comm_csv = csv.writer(comm_io)
    # 添加 csv 头部信息
    comm_csv.writerow(["uid","username","isvip","ip","city","content","ctime","like","replynum","source","title","subtitle","play_count"])
    episodes_data = get_episode(episode_id,episode_nums)
    episodes = episodes_data["data"]["episode"]["videolist"]
    for episode in tqdm(episodes):
        vid = episode["vid"]
        base_info = get_base_info(vid)
        vcomm_count = base_info["vcomm_count"]
        # 一次最多 60 条
        for i in range((vcomm_count//60) + 1 ):
            comms = comm_get(60, i + 1,vid,pid)
            parse_comm(comm_csv,comms["data"],episode)
    
# 创建存储弹幕信息的文件
comm = open(r'/Users/weduoo/Desktop/hls_comm.csv', 'a')
create_comm(comm,29394106,55,10031905)
comm.close()

In [ ]:
def create_dm(dm_io,episode_id,episode_nums):
    dm_csv = csv.writer(dm_io)
    # 添加 csv 头部信息
    dm_csv.writerow(["uid","addtime","version","txt","type","title","subtitle"])
    episodes_data = get_episode(episode_id,episode_nums)
    episodes = episodes_data["data"]["episode"]["videolist"]
    for episode in tqdm(episodes):
        vid = episode["vid"]
        base_info = get_base_info(vid)
        vdm_count = base_info["vdm_count"]
        
        # 一次最多 60 条
        for i in range(0,25,3):
            dms = dm_get(i * 100,vid)
            parse_dm(dm_csv, dms["data"]["list"], episode)
    
# 创建存储弹幕信息的文件
dm = open(r'/Users/weduoo/Desktop/hls_dm.csv', 'a')
create_dm(dm,29394106,55)
dm.close()

## 弹幕分析

In [ ]:
data = pd.read_csv("/Users/weduoo/Desktop/hls_dm.csv")
data.head()

In [ ]:
y = data.groupby("title").size()
bar = Bar("每集弹幕数量",width=950)
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
import re
dm_text = [] 
def remove_words(txt):
    words = ["哈","呵","啦","啊","皇上","甄","嬛","弹幕呢"]
    for word in words:
        txt = txt.replace(word,"")
    return txt
    
for text in tqdm(list(data["txt"])):
    t = remove_words(str(text))
    t = re.sub('[a-zA-Z]','', t)
    dm_text.append(t.strip())
create_wordcloud("\n".join(dm_text))

## 评论分析

In [ ]:
comms = pd.read_csv("/Users/weduoo/Desktop/hls_comm.csv")
comms.head()

In [ ]:
y = comms.groupby("title").size()
bar = Bar("每集评论数量",width=950)
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
citys = comms["city"].str.replace("省","")
citys = citys.str.replace("北京市","北京")
# citys = citys[citys.isin(provinces)]
y = citys.value_counts()
bar = Bar( str(len(y.index)) + "个地区评论数分布",width=950)
print(list(y.index))
bar.add("Total",list(y.index), y, is_label_show=True, 
        is_datazoom_show=True, datazoom_type='both',
        datazoom_range=[10, 25])
bar

In [ ]:
import re
comms_text = [] 
def remove_words(txt):
    words = ["哈","呵","啦","啊","皇上","甄","嬛","弹幕呢"]
    for word in words:
        txt = txt.replace(word,"")
    return txt
    
for text in tqdm(list(comms["content"])):
    t = remove_words(str(text))
    t = re.sub('[a-zA-Z]','', t)
    comms_text.append(t.strip())
create_wordcloud("\n".join(comms_text))

# 测试

In [ ]:
import re
s='~!@#adjdakdakjjkd002212ADSAADA()//../eqqwedadjadj'
print(re.sub('[A-Za-z]','',s))

In [ ]:
from urllib import request
@retry
def try_timeout():
    url = "http://api.my.le.com/vcm/api/list?type=video&rows=20&page=84&sort=&cid=2&\
    source=1&xid=29394106&pid=10031905&listType=1"
    response = request.urlopen(url)
    response = response.read().decode('utf-8')
    data = json.loads(response)
    print(len(data["data"]))
try_timeout()

In [ ]:
5038//60

In [ ]:
from urllib import request
@retry
def try_timeout():
    url = "http://cdn.api.my.le.com/danmu/list?vid=1566616&cid=2&start=300&amount=2000"
    response = request.urlopen(url)
    response = response.read().decode('utf-8')
    data = json.loads(response)
    print(len(data["data"]["list"]))
try_timeout()

In [ ]:
import re
 
content = 'jQuery19105577507525270364_1525079352904({"result":"200","total":7656,"replyTotal":2285,"data":"(123)"})'
result = re.findall('\((.+)\\)', content)
print(result)

In [ ]:
d = {'name':"Tom", 'age':10, 'Tel':110}  
#打印返回值  
print('name' in d.keys())

In [ ]:
25675//60

In [ ]:
for i in range(0,25,3):
    print(i)

In [ ]:
import random
from retrying import retry

@retry
def do_something_unreliable():
    ri = random.randint(0, 10)
    print(ri)
    if ri > 1:
        raise IOError("Broken sauce, everything is hosed!!!111one")
    else:
        return "Awesome sauce!"

print(do_something_unreliable())

In [ ]:
1250000//76